In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count

In [2]:
spark = SparkSession.builder.appName("SalesData").getOrCreate()

In [3]:
sales_file = "../data/sales_records.csv"
df = spark.read.csv(sales_file, header=True, inferSchema=True)

In [4]:
df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(sales_file)

In [5]:
df.count()

100000

In [6]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[Region: string, Country: string, Item Type: string, Sales Channel: string, Order Priority: string, Order Date: string, Order ID: int, Ship Date: string, Units Sold: int, Unit Price: double, Unit Cost: double, Total Revenue: double, Total Cost: double, Total Profit: double]>

In [7]:
#df.select("Region", "Country", "Order ID").show(n=10, truncate=True)
df.select("Region", "Country", "Order ID").show(n=10, truncate=False)

+---------------------------------+---------------------+---------+
|Region                           |Country              |Order ID |
+---------------------------------+---------------------+---------+
|Middle East and North Africa     |Azerbaijan           |535113847|
|Central America and the Caribbean|Panama               |874708545|
|Sub-Saharan Africa               |Sao Tome and Principe|854349935|
|Sub-Saharan Africa               |Sao Tome and Principe|892836844|
|Central America and the Caribbean|Belize               |129280602|
|Europe                           |Denmark              |473105037|
|Europe                           |Germany              |754046475|
|Middle East and North Africa     |Turkey               |772153747|
|Europe                           |United Kingdom       |847788178|
|Asia                             |Kazakhstan           |471623599|
+---------------------------------+---------------------+---------+
only showing top 10 rows



In [8]:
### Aggregated Results
count_by_country_df = df.groupBy("Region", "Country").count()
count_by_country_df.orderBy("count", ascending=False).show(5, truncate=False)

+---------------------+------------+-----+
|Region               |Country     |count|
+---------------------+------------+-----+
|Sub-Saharan Africa   |Sudan       |623  |
|Australia and Oceania|New Zealand |593  |
|Europe               |Vatican City|590  |
|Europe               |Malta       |589  |
|Sub-Saharan Africa   |Mozambique  |589  |
+---------------------+------------+-----+
only showing top 5 rows



In [9]:
count_sales_df = (df.select("Region", "Country", "Order ID")
                  .groupBy("Region", "Country")
                  .agg(count("Order ID").alias("Total Orders"))
                  .orderBy("Total Orders", ascending=False)
                 )

count_sales_df.show(10, truncate=False)
print("Total order = ", count_sales_df.count())



+---------------------------------+-------------+------------+
|Region                           |Country      |Total Orders|
+---------------------------------+-------------+------------+
|Sub-Saharan Africa               |Sudan        |623         |
|Australia and Oceania            |New Zealand  |593         |
|Europe                           |Vatican City |590         |
|Europe                           |Malta        |589         |
|Sub-Saharan Africa               |Mozambique   |589         |
|Middle East and North Africa     |Tunisia      |584         |
|Asia                             |Cambodia     |584         |
|Central America and the Caribbean|Panama       |578         |
|Sub-Saharan Africa               |Rwanda       |576         |
|Sub-Saharan Africa               |Cote d'Ivoire|575         |
+---------------------------------+-------------+------------+
only showing top 10 rows

Total order =  185
